In the real world, even a profitable strategy can be too risky.
The Risk Agent is like a compliance officer: it looks at the strategy’s performance and checks if it’s within safe limits.

We are creating a RiskAgent class.

__init__ stores the thresholds we want (like Sharpe minimum).

assess takes strategy metrics and checks if they are safe.

It outputs both a decision (pass/fail) and a human-readable explanation.

In [1]:
class RiskAgent:
    def __init__(self, sharpe_min=1.0, max_dd=-0.25, vol_max=0.3):
        """
        sharpe_min: minimum acceptable Sharpe ratio
        max_dd: maximum allowed drawdown (e.g. -0.25 = -25%)
        vol_max: maximum annualized volatility
        """
        self.sharpe_min = sharpe_min
        self.max_dd = max_dd
        self.vol_max = vol_max
    
    def assess(self, metrics: dict):
        """
        Takes a metrics dictionary {CAGR, Volatility, Sharpe, MaxDD}
        Returns pass/fail decision and notes
        """
        notes = []
        decision = "PASS"
        
        if metrics["Sharpe"] < self.sharpe_min:
            notes.append(f"Sharpe too low ({metrics['Sharpe']:.2f} < {self.sharpe_min})")
            decision = "FAIL"
        
        if metrics["MaxDD"] < self.max_dd:
            notes.append(f"Drawdown too high ({metrics['MaxDD']:.2%} < {self.max_dd:.0%})")
            decision = "FAIL"
        
        if metrics["Volatility"] > self.vol_max:
            notes.append(f"Volatility too high ({metrics['Volatility']:.2%} > {self.vol_max:.0%})")
            decision = "FAIL"
        
        if not notes:
            notes.append("All risk checks passed")
        
        return {"Decision": decision, "Notes": notes}


We now connect the two agents:

StrategyAgent produces results (returns, metrics).

RiskAgent checks if those results are acceptable.
This is the first step in orchestration.

In [4]:
import sys
sys.path.append("../src")

from strategy_agent import StrategyAgent
import pandas as pd
import numpy as np

# Load dataset
data = pd.read_csv("../data/AAPL_enriched.csv", index_col=0, parse_dates=True)

# Run SMA strategy
agent = StrategyAgent(data)
sma_res = agent.sma_crossover()

# Compute metrics
def compute_metrics(df):
    strat_curve = (1 + df["Strategy_Return"]).cumprod()
    years = (df.index[-1] - df.index[0]).days / 365.25
    cagr = strat_curve.iloc[-1]**(1/years) - 1
    vol = df["Strategy_Return"].std() * np.sqrt(252)
    sharpe = (df["Strategy_Return"].mean() * 252) / (df["Strategy_Return"].std() * np.sqrt(252))
    roll_max = strat_curve.cummax()
    dd = (strat_curve / roll_max - 1).min()
    return {"CAGR": cagr, "Volatility": vol, "Sharpe": sharpe, "MaxDD": dd}

metrics = compute_metrics(sma_res)

# Assess with Risk Agent
risk = RiskAgent(sharpe_min=1.0, max_dd=-0.25, vol_max=0.3)
assessment = risk.assess(metrics)

print("Metrics:", metrics)
print("Assessment:", assessment)


Metrics: {'CAGR': np.float64(-0.12185634208588725), 'Volatility': np.float64(0.04124301625654238), 'Sharpe': np.float64(-3.175535679227622), 'MaxDD': np.float64(-0.026329927975329626)}
Assessment: {'Decision': 'FAIL', 'Notes': ['Sharpe too low (-3.18 < 1.0)']}


We export the RiskAgent to a file in /src/, just like StrategyAgent.
This lets us reuse it in later orchestration (Allocator Agent, full pipelines).

In [5]:
with open("../src/risk_agent.py", "w") as f:
    f.write("""class RiskAgent:
    def __init__(self, sharpe_min=1.0, max_dd=-0.25, vol_max=0.3):
        self.sharpe_min = sharpe_min
        self.max_dd = max_dd
        self.vol_max = vol_max
    
    def assess(self, metrics: dict):
        notes = []
        decision = "PASS"
        
        if metrics["Sharpe"] < self.sharpe_min:
            notes.append(f"Sharpe too low ({metrics['Sharpe']:.2f} < {self.sharpe_min})")
            decision = "FAIL"
        
        if metrics["MaxDD"] < self.max_dd:
            notes.append(f"Drawdown too high ({metrics['MaxDD']:.2%} < {self.max_dd:.0%})")
            decision = "FAIL"
        
        if metrics["Volatility"] > self.vol_max:
            notes.append(f"Volatility too high ({metrics['Volatility']:.2%} > {self.vol_max:.0%})")
            decision = "FAIL"
        
        if not notes:
            notes.append("All risk checks passed")
        
        return {"Decision": decision, "Notes": notes}""")
